이용 건수는 중복값을 처리해서 계산하면 안된다.
-> 해당월의 총 이용건수를 봐야하기 때문에.

이용인원만 중복값을 처리한 후 계산한다. 


# 1, 엑셀 데이터 통합하기 
## 받아야하는 데이터
### 1. 일반+위기
### 2. 상주
### 3. 고위험군
### 4. 특별개임 프로그램


In [4]:
df = pd.read_excel("C:\workshop\data\고객사 직원 상담 리스트.xls", header = 1)
df

ValueError: File is not a recognized excel file

In [5]:
import openpyxl
import pandas as pd
import numpy as np

#데이터 프레임 만들기
df_일반 = pd.read_excel("C:\workshop\data\국민은행 일반_1월 데이터.xlsx", header = 1)
# df_위기 = pd.read_excel("C:\workshop\data\", header = 1)
# df_연계 = pd.concat([df_일반, df_위기])                     
df_연계 = df_일반

df_상주_서울 = pd.read_excel("C:\workshop\data\국민은행_상주_서울_1월.xlsx", header = 1)
df_상주_대전 = pd.read_excel("C:\workshop\data\국민은행_상주 대전_1월.xlsx", header = 1)             
# df_고위험군 = pd.read_excel("C:\workshop\data\", header = 1)
# df_특별개입 = pd.read_excel("C:\workshop\data\", header = 1)
                        
#모든 데이터 합치기                         
# df_og = pd.concat([df_일반, df_위기, df_상주_서울, df_상주_대전, df_고위험군, df_특별개입])
df_og = pd.concat([df_일반, df_상주_서울, df_상주_대전])                      

ValueError: File is not a recognized excel file

In [9]:
#데이터 프레임들에서 필요한 행과 컬럼만 남긴다. 

# df_list = ['df_일반', 'df_위기', 'df_상주', 'df_고위험군', 'df_특별개입', 'df_og']                        
# df_list = ['df_일반', 'df_위기', 'df_상주_서울', 'df_상주_대전']                        
df_list = [df_일반]                        


for df_ex in df_list:
    df_ex = df_ex.loc[:,['내담자명', '전화번호', '상담신청경로', '상담주제','상담분야','상담이용대상', '성별', '연령', 
                     '직위','지역', '효과성 검사 위험도 수준', '참석동반자','상담방법', '상담목표','상담사', '상담내용', 
                     '상담회차','상담날짜','상담시작시간','상담종료시간','진행시간','참석여부', '상담일지등록일', 
                     '기본상담료','대표자격증','급수','소속기관']]
    
    #nan값 있는 0번째 인덱스 행 지우기 
    df_ex = df_ex.drop(index = 0, axis = 0)
    
    #상담일지 등록일을 기준으로 월보고를 작성한다. (확인필요)
    #먼저 날짜 컬럼을 스트링으로 변경한다. 
    df_ex['상담일지등록일']= df_ex['상담일지등록일'].astype('str')
    #0000-00-00형식의 날짜로 변환한다. 
    df_ex['상담일지등록일'] = pd.to_datetime(df_ex['상담일지등록일'],format='%Y-%m-%d')
    #해당 날짜에 해당하는 값들 추출한다. 
    df_ex = df_ex[df_ex['상담일지등록일'].between('2022-01-01', '2022-01-31')]

    #참석 동반자 결측값 직원으로 채우기
    #df_ex['참석동반자'] = df_ex['참석동반자'].replace(to_replace = np.nan, value = '직원')

    ## 이용인원구하기 위한 데이터프레임 
    #마지막만 남기고 중복값 지우기 (동명이인일 경우를 생각해서 전화번호를 고유값으로 정한다.)
    #마지막 데이터를 남기는 이유는 : 해당 내담자가 진행한 마지막 회기(최신상태)만 남기기 위해서
    df_ex_people = df_ex.drop_duplicates(['전화번호'], keep='last')

    #target_month = 1
    # df = df[df['상담일지등록일'].dt.month==target_month] 
    # 또는 df1 = df1.query('상담일지등록일.dt.month == @target_month')

    # 인덱스 리셋하기
    df_ex = df_ex.reset_index(drop = True) # 상담 이용건수 구하는 dataframe
    df_ex_people = df_ex_people.reset_index(drop = True) #이용인원 구하는 dataframe

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['급수'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

# 운영개요

# 1. 이용실적

In [15]:
# df_people['내담자명'].value_counts()
len(df_people['내담자명'])

55

In [18]:
# df['내담자명'].value_counts()
len(df['내담자명'])

137

In [4]:
print('이용인원에 따른 상담방법',df_people['상담방법'].value_counts())
print('')
print('이용건수에 따른 상담방법',df['상담방법'].value_counts())

이용인원에 따른 상담방법 대면    53
전화     1
화상     1
Name: 상담방법, dtype: int64

이용건수에 따른 상담방법 대면    135
전화      1
화상      1
Name: 상담방법, dtype: int64


In [20]:
df_people['상담방법'].value_counts('전화')

대면    0.963636
전화    0.018182
화상    0.018182
Name: 상담방법, dtype: float64

In [5]:
#### 1. 이용실적 표 구하기 
print("2페이지")
# 대면상담 중 가족 이용인원
df_inperson_family_people = df_people[(df_people['상담방법'] == '대면') & (df_people["참석동반자"].str.startswith("가족"))]
df_inperson_family_people = df_inperson_family_people.reset_index()
df_inperson_family_people_num = len(df_inperson_family_people)
print("대면상담 중 가족 이용인원 : ", df_inperson_family_people_num)

# 대면상담 중 가족 이용건수 
df_inperson_family_counts = df[(df['상담방법'] == '대면') & (df["참석동반자"].str.startswith("가족"))]
df_inperson_family_counts = df_inperson_family_counts.reset_index()
df_inperson_family_counts_num = len(df_inperson_family_counts)
print("대면상담 중 가족 이용건수 : ", df_inperson_family_counts_num )

# 대면상담 중 직원 이용인원
df_inperson_employee_people = df_people[(df_people['상담방법'] == '대면') & (df_people["참석동반자"].str.startswith("직원"))]
df_inperson_employee_people = df_inperson_employee_people.reset_index()
df_inperson_employee_people_num = len(df_inperson_employee_people)
print("대면상담 중 직원 이용인원 : ", df_inperson_employee_people_num )

# 대면상담 중 직원 이용건수 
df_inperson_employee_counts = df[(df['상담방법'] == '대면') & (df["참석동반자"].str.startswith("직원"))]
df_inperson_employee_counts = df_inperson_employee_counts.reset_index()
df_inperson_employee_counts_num = len(df_inperson_employee_counts)
print("대면상담 중 직원 이용건수 : ", df_inperson_employee_counts_num )

# 전화상담 중 가족 이용인원
df_call_family_people = df_people[(df_people['상담방법'] == '전화') & (df_people["참석동반자"].str.startswith("가족"))]
df_call_family_people = df_call_family_people.reset_index()
df_call_family_people_num = len(df_call_family_people)
print("전화상담 중 가족 이용인원 : ", df_call_family_people_num)


# 전화상담 중 가족 이용건수 
df_call_family_counts = df[(df['상담방법'] == '전화') & (df["참석동반자"].str.startswith("가족"))]
df_call_family_counts = df_call_family_counts.reset_index()
df_call_family_counts_num = len(df_call_family_counts)
print("전화상담 중 가족 이용건수 : ", df_call_family_counts_num)

# 전화상담 중 직원 이용인원
df_call_employee_people = df_people[(df_people['상담방법'] == '전화') & (df_people["참석동반자"].str.startswith("직원"))]
df_call_employee_people = df_call_employee_people.reset_index()
df_call_employee_people_num = len(df_call_employee_people)
print("전화상담 중 직원 이용인원 : ", df_call_employee_people_num)


# 전화상담 중 직원 이용건수 
df_call_employee_counts = df[(df['상담방법'] == '전화') & (df["참석동반자"].str.startswith("직원"))]
df_call_employee_counts = df_call_employee_counts.reset_index()
df_call_employee_counts_num = len(df_call_employee_counts)
print("전화상담 중 직원 이용건수 : ", df_call_employee_counts_num)


# 화상상담 중 가족 이용인원
df_zoom_family_people = df_people[(df_people['상담방법'] == '화상') & (df_people["참석동반자"].str.startswith("가족"))]
df_zoom_family_people = df_zoom_family_people.reset_index()
df_zoom_family_people_num = len(df_zoom_family_people)
print("화상상담 중 가족 이용인원 : ", df_zoom_family_people_num)

# 화상상담 중 가족 이용건수 
df_zoom_family_counts = df[(df['상담방법'] == '화상') & (df["참석동반자"].str.startswith("가족"))]
df_zoom_family_counts = df_zoom_family_counts.reset_index()
df_zoom_family_counts_num = len(df_zoom_family_counts)
print("화상상담 중 가족 이용건수 : ", df_zoom_family_counts_num)

# 화상상담 중 직원 이용인원
df_zoom_employee_people = df_people[(df_people['상담방법'] == '화상') & (df_people["참석동반자"].str.startswith("직원"))]
df_zoom_employee_people = df_zoom_employee_people.reset_index()
df_zoom_employee_people_num = len(df_zoom_employee_people)
print("화상상담 중 직원 이용인원 : ", df_zoom_employee_people_num)

# 화상상담 중 직원 이용건수 
df_zoom_employee_counts = df[(df['상담방법'] == '화상') & (df["참석동반자"].str.startswith("직원"))]
df_zoom_employee_counts = df_zoom_employee_counts.reset_index()
df_zoom_employee_counts_num = len(df_zoom_employee_counts)
print("화상상담 중 직원 이용건수 : ", df_zoom_employee_counts_num)

# 이메일 중 가족 이용인원
df_email_family_people = df_people[(df_people['상담방법'] == '이메일') & (df_people["참석동반자"].str.startswith("가족"))]
df_email_family_people = df_email_family_people.reset_index()
df_email_family_people_num = len(df_email_family_people)
print("이메일 중 가족 이용인원 : ", df_email_family_people_num)

# 이메일 중 가족 이용건수 
df_email_family_counts = df[(df['상담방법'] == '이메일') & (df["참석동반자"].str.startswith("가족"))]
df_email_family_counts = df_email_family_counts.reset_index()
df_email_family_counts_num = len(df_email_family_counts)
print("이메일 중 가족 이용건수  : ", df_email_family_counts_num)

# 이메일 중 직원 이용인원
df_email_employee_people = df_people[(df_people['상담방법'] == '이메일') & (df_people["참석동반자"].str.startswith("직원"))]
df_email_employee_people = df_email_employee_people.reset_index()
df_email_employee_people_num = len(df_email_employee_people)
print("이메일 중 직원 이용인원 : ", df_email_employee_people_num)


# 이메일 중 직원 이용건수 
df_email_employee_counts = df[(df['상담방법'] == '이메일') & (df["참석동반자"].str.startswith("직원"))]
df_email_employee_counts = df_email_employee_counts.reset_index()
df_email_employee_counts_num = len(df_email_employee_counts)
print("이메일 중 직원 이용건수 : ", df_email_employee_counts_num)

# 톡상담 중 가족 이용인원
df_talk_family_people = df_people[(df_people['상담방법'] == '톡상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_talk_family_people = df_email_family_people.reset_index()
df_talk_family_people_num = len(df_talk_family_people)
print("톡상담 중 가족 이용인원 : ", df_talk_family_people_num)

# 톡상담 중 가족 이용건수 
df_talk_zoom_family_counts = df[(df['상담방법'] == '톡상담') & (df["참석동반자"].str.startswith("가족"))]
df_talk_zoom_family_counts = df_talk_zoom_family_counts.reset_index()
df_talk_zoom_family_counts_num = len(df_talk_zoom_family_counts)
print("톡상담 중 가족 이용건수 : ", df_talk_zoom_family_counts_num)

# 톡상담 중 직원 이용인원
df_talk_zoom_employee_people = df_people[(df_people['상담방법'] == '톡상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_talk_zoom_employee_people = df_talk_zoom_employee_people.reset_index()
df_talk_zoom_employee_people_num = len(df_talk_zoom_employee_people)
print("톡상담 중 직원 이용인원 : ", df_talk_zoom_employee_people_num)

# 톡상담 중 직원 이용건수 
df_talk_employee_counts = df[(df['상담방법'] == '톡상담') & (df["참석동반자"].str.startswith("직원"))]
df_talk_employee_counts = df_talk_employee_counts.reset_index()
df_talk_employee_counts_num = len(df_talk_employee_counts)
print("톡상담 중 직원 이용건수 : ", df_talk_employee_counts_num)


# 야간상담 중 가족 이용인원
df_night_family_people = df_people[(df_people['상담방법'] == '야간상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_night_family_people = df_night_family_people.reset_index()
df_night_family_people_num = len(df_night_family_people)
print("야간상담 중 가족 이용인원 : ", df_night_family_people_num)

# 야간상담 중 가족 이용건수 
df_night_zoom_family_counts = df[(df['상담방법'] == '야간상담') & (df["참석동반자"].str.startswith("가족"))]
df_night_zoom_family_counts = df_night_zoom_family_counts.reset_index()
df_night_zoom_family_counts_num = len(df_night_zoom_family_counts)
print("야간상담 중 가족 이용건수 : ", df_night_zoom_family_counts_num)

# 야간상담 중 직원 이용인원
df_night_zoom_employee_people = df_people[(df_people['상담방법'] == '야간상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_night_zoom_employee_people = df_night_zoom_employee_people.reset_index()
df_night_zoom_employee_people_num = len(df_night_zoom_employee_people)
print("야간상담 중 직원 이용인원 : ", df_night_zoom_employee_people_num)

# 야간상담 중 직원 이용건수 
df_night_employee_counts = df[(df['상담방법'] == '야간상담') & (df["참석동반자"].str.startswith("직원"))]
df_night_employee_counts = df_night_employee_counts.reset_index()
df_night_employee_counts_num = len(df_night_employee_counts)
print("야간상담 중 직원 이용건수 : ", df_night_employee_counts_num)

# 상주상담 중 가족 이용인원
df_stay_family_people = df_people[(df_people['상담방법'] == '상주상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_stay_family_people = df_stay_family_people.reset_index()
df_stay_family_people_num = len(df_stay_family_people)
print("상주상담 중 가족 이용인원 : ", df_stay_family_people_num)

# 상주상담 중 가족 이용건수 
df_stay_zoom_family_counts = df[(df['상담방법'] == '상주상담') & (df["참석동반자"].str.startswith("가족"))]
df_stay_zoom_family_counts = df_stay_zoom_family_counts.reset_index()
df_stay_zoom_family_counts_num = len(df_stay_zoom_family_counts)
print("상주상담 중 가족 이용건수 : ", df_stay_zoom_family_counts_num)


# 상주상담 중 직원 이용인원
df_stay_zoom_employee_people = df_people[(df_people['상담방법'] == '상주상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_stay_zoom_employee_people = df_stay_zoom_employee_people.reset_index()
df_stay_zoom_employee_people_num = len(df_stay_zoom_employee_people)
print("상주상담 중 직원 이용인원 : ", df_stay_zoom_employee_people_num)

# 상주상담 중 직원 이용건수 
df_stay_employee_counts = df[(df['상담방법'] == '상주상담') & (df["참석동반자"].str.startswith("직원"))]
df_stay_employee_counts = df_stay_employee_counts.reset_index()
df_stay_employee_counts_num = len(df_stay_employee_counts)
print("상주상담 중 직원 이용건수 : ", df_stay_employee_counts_num)

# 집단상담 중 가족 이용인원
df_group_family_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_group_family_people = df_group_family_people.reset_index()
df_group_family_people_num = len(df_group_family_people)
print("집단상담 중 가족 이용인원 : ", df_group_family_people_num)

# 집단상담 중 가족 이용건수 
df_group_family_counts = df[(df['상담방법'] == '집단상담') & (df["참석동반자"].str.startswith("가족"))]
df_group_family_counts = df_group_family_counts.reset_index()
df_group_family_counts_num = len(df_group_family_counts)
print("집단상담 중 가족 이용건수 : ", df_group_family_counts_num)

# 집단상담 중 직원 이용인원
df_group_employee_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("직원"))]
df_group_employee_people = df_group_employee_people.reset_index()
df_group_employee_people_num = len(df_group_employee_people)
print("집단상담 중 직원 이용인원 : ", df_group_employee_people_num)

# 집단상담 중 직원 이용건수 
df_group_employee_counts = df[(df['상담방법'] == '집단상담') & (df["참석동반자"].str.startswith("직원"))]
df_group_employee_counts = df_group_employee_counts.reset_index()
df_group_employee_counts_num = len(df_group_employee_counts)
print("집단상담 중 직원 이용건수 : ", df_group_employee_counts_num)

# 노쇼 중 가족 이용인원
df_noshow_family_people = df_people[(df_people['상담방법'] == '집단상담') & (df_people["참석동반자"].str.startswith("가족"))]
df_noshow_family_people = df_noshow_family_people.reset_index()
df_noshow_family_people_num = len(df_noshow_family_people)
print("집단상담 중 가족 이용인원 : ", df_noshow_family_people_num)

# 노쇼 중 가족 이용건수 
df_noshow_zoom_family_counts = df[(df['상담방법'] == '노쇼') & (df["참석동반자"].str.startswith("가족"))]
df_noshow_zoom_family_counts = df_noshow_zoom_family_counts.reset_index()
df_noshow_zoom_family_counts_num = len(df_noshow_zoom_family_counts)
print("노쇼 중 가족 이용건수 : ", df_noshow_zoom_family_counts_num)

# 노쇼 중 직원 이용인원
df_noshow_zoom_employee_people = df_people[(df_people['상담방법'] == '노쇼') & (df_people["참석동반자"].str.startswith("직원"))]
df_noshow_zoom_employee_people = df_noshow_zoom_employee_people.reset_index()
df_noshow_zoom_employee_people_num = len(df_noshow_zoom_employee_people)
print("노쇼 중 직원 이용인원 : ", df_noshow_zoom_employee_people_num)

# 노쇼 중 직원 이용건수 
df_noshow_employee_counts = df[(df['상담방법'] == '노쇼') & (df["참석동반자"].str.startswith("직원"))]
df_noshow_employee_counts = df_noshow_employee_counts.reset_index()
df_noshow_employee_counts_num = len(df_noshow_employee_counts)
print("노쇼 중 직원 이용건수 : ", df_noshow_employee_counts_num)

2페이지
대면상담 중 가족 이용인원 :  23
대면상담 중 가족 이용건수 :  64
대면상담 중 직원 이용인원 :  30
대면상담 중 직원 이용건수 :  71
전화상담 중 가족 이용인원 :  0
전화상담 중 가족 이용건수 :  0
전화상담 중 직원 이용인원 :  1
전화상담 중 직원 이용건수 :  1
화상상담 중 가족 이용인원 :  1
화상상담 중 가족 이용건수 :  1
화상상담 중 직원 이용인원 :  0
화상상담 중 직원 이용건수 :  0
이메일 중 가족 이용인원 :  0
이메일 중 가족 이용건수  :  0
이메일 중 직원 이용인원 :  0
이메일 중 직원 이용건수 :  0
톡상담 중 가족 이용인원 :  0
톡상담 중 가족 이용건수 :  0
톡상담 중 직원 이용인원 :  0
톡상담 중 직원 이용건수 :  0
야간상담 중 가족 이용인원 :  0
야간상담 중 가족 이용건수 :  0
야간상담 중 직원 이용인원 :  0
야간상담 중 직원 이용건수 :  0
상주상담 중 가족 이용인원 :  0
상주상담 중 가족 이용건수 :  0
상주상담 중 직원 이용인원 :  0
상주상담 중 직원 이용건수 :  0
집단상담 중 가족 이용인원 :  0
집단상담 중 가족 이용건수 :  0
집단상담 중 직원 이용인원 :  0
집단상담 중 직원 이용건수 :  0
집단상담 중 가족 이용인원 :  0
노쇼 중 가족 이용건수 :  0
노쇼 중 직원 이용인원 :  0
노쇼 중 직원 이용건수 :  0


# 2. 월별이용현황

In [6]:
print('월별이용현황, 이용인원')
print(df_people['참석동반자'].value_counts())
print('')
print('월별이용현황, 이용건수')
print(df['참석동반자'].value_counts())
print('')

월별이용현황, 이용인원
직원          31
가족 - 자녀     19
가족 - 배우자     3
가족 - 기타      2
Name: 참석동반자, dtype: int64

월별이용현황, 이용건수
직원          72
가족 - 자녀     47
가족 - 배우자    10
가족 - 기타      8
Name: 참석동반자, dtype: int64



# 3. 인적사항

In [7]:
print("인적사항_성별 구분, 이용인원")
print(df_people['성별'].value_counts())
print('')
print("인적사항_성별 구분, 이용건수")
print(df['성별'].value_counts())
print('')
print("인적사항_연령별 구분, 이용인원")
print(df_people['연령'].value_counts())
print('')
print("인적사항_연령별 구분, 이용건수")
print(df['연령'].value_counts())
print('')
# print("인적사항_직군별 구분, 이용인원")
# print(df_people['직군'].value_counts())
# print('')
# print("인적사항_직군별 구분, 이용건수")
# print(df['직군'].value_counts())
# print('')
print("인적사항_직위별 구분, 이용인원")
print(df_people['직위'].value_counts())
print('')
print("인적사항_직위별 구분, 이용건수")
print(df['직위'].value_counts())
print('')
print("인적사항_지역별 구분, 이용인원")
print(df_people['지역'].value_counts())
print('')
print("인적사항_지역별 구분, 이용건수")
print(df['지역'].value_counts())
print('')

인적사항_성별 구분, 이용인원
여    39
남    16
Name: 성별, dtype: int64

인적사항_성별 구분, 이용건수
여    87
남    50
Name: 성별, dtype: int64

인적사항_연령별 구분, 이용인원
40대    27
30대    15
50대    12
20대     1
Name: 연령, dtype: int64

인적사항_연령별 구분, 이용건수
40대    67
30대    36
50대    31
20대     3
Name: 연령, dtype: int64

인적사항_직위별 구분, 이용인원
대리                      18
과장 차장                   18
임원 전문직무직원 임금피크 파트타이머     7
계장                       6
팀장 부장 등                  6
Name: 직위, dtype: int64

인적사항_직위별 구분, 이용건수
과장 차장                   55
대리                      39
임원 전문직무직원 임금피크 파트타이머    22
팀장 부장 등                 15
계장                       6
Name: 직위, dtype: int64

인적사항_지역별 구분, 이용인원
서울          32
경기          10
부산 울산 경남     7
대전 세종 충남     3
대구 경북        2
인천           1
Name: 지역, dtype: int64

인적사항_지역별 구분, 이용건수
서울          84
경기          22
부산 울산 경남    18
대전 세종 충남     7
대구 경북        5
인천           1
Name: 지역, dtype: int64



# 4. 이용자 현황

In [8]:
print("4. 이용자 현황_월별 신규인원")
df신규인원 = df['상담회차'] == '1회차'
## flase를 세면됨 
print(df신규인원.value_counts())
print("")

print("4. 이용자 현황_상담신청 경로")
print(df_people['상담신청경로'].value_counts())
print("notice : 기타는 의뢰입니다")
print("")

print("4. 이용자 현황_내담자 마음건강 상태")
print(df_people['효과성 검사 위험도 수준'].value_counts())

# df[(df['상담방법'] == '집단상담') & (df["참석동반자"].str.startswith("가족"))]
# 상담주제, 효과성 검사 위험도 수준

4. 이용자 현황_월별 신규인원
False    80
True     57
Name: 상담회차, dtype: int64

4. 이용자 현황_상담신청 경로
홍보레터           32
기타CUSCUS099    18
지인추천            3
프로그램            1
설명회             1
Name: 상담신청경로, dtype: int64
notice : 기타는 의뢰입니다

4. 이용자 현황_내담자 마음건강 상태
무응답    18
예방군    12
주의군     8
안정군     8
보호군     6
건강군     3
Name: 효과성 검사 위험도 수준, dtype: int64


# 분석결과

# 1. EAP 상담 진행 내용

In [9]:
print("1) 회기별 분류")
print(df_people['상담회차'].value_counts())
print("")

print("2) 상담분야별분류_이용인원")
print(df_people['상담분야'].value_counts())
print("")

print("2) 상담분야별분류_이용건수")
print(df['상담분야'].value_counts())
print("")

print("[개인/조직 상담 total]")
print("False : 개인, True :  조직")
df_개인 = df_people['상담분야'].str.startswith("Work")
print(df_개인.value_counts())

df_people['참석동반자'] == '직원'

1) 회기별 분류
1회차    18
2회차    14
3회차    13
4회차     6
6회차     2
7회차     1
5회차     1
Name: 상담회차, dtype: int64

2) 상담분야별분류_이용인원
자녀         32
우울          8
업무스트레스      3
배우자         3
스트레스관리      2
기타          2
원가족         2
대인관계        1
조직내관계갈등     1
성격          1
Name: 상담분야, dtype: int64

2) 상담분야별분류_이용건수
자녀         81
우울         12
업무스트레스     11
배우자        10
원가족         7
기타          5
스트레스관리      4
대인관계        3
조직내관계갈등     3
성격          1
Name: 상담분야, dtype: int64

[개인/조직 상담 total]
False : 개인, True :  조직
False    55
Name: 상담분야, dtype: int64


0     False
1      True
2      True
3      True
4     False
5      True
6     False
7      True
8      True
9      True
10    False
11     True
12    False
13    False
14     True
15     True
16     True
17     True
18     True
19     True
20    False
21    False
22     True
23    False
24    False
25    False
26     True
27    False
28     True
29     True
30     True
31    False
32     True
33    False
34     True
35    False
36     True
37    False
38     True
39    False
40    False
41     True
42    False
43    False
44    False
45     True
46     True
47    False
48     True
49    False
50     True
51     True
52    False
53     True
54     True
Name: 참석동반자, dtype: bool

In [10]:
df.to_excel("C:\workshop\국민은행_일반_상담분야확인용.xlsx")

In [11]:
df_people.to_excel("C:\workshop\국민은행_일반_수준확인용.xlsx")

In [12]:
#데이터 현황 파악하기 
#people = []
#count_p = 0
#count_c = 0

#for i in df['전화번호']:
#    count_c += 1
#    if i in people:
#        pass
#    else:
#        people.append(i)
#        count_p += 1

#print("내담자 수 :", count_p)
#print("상담건수 :", count_c)
#print("내담자 이름 :", people)

In [13]:
#데이터 현황 파악하기 
#ppl = []
#counts = 0
#counsel = 0

#for p in df['내담자명']:
#    counsel += 1
#    if p in ppl:
#        pass
#    else:
#        ppl.append(p)
#         ppl.remove(nan)
#        counts += 1
    
#print('내담자 수 :',counts)
#print('상담건수 :', counsel)
#print('내담자 이름 :', ppl)

In [14]:
#1월달 데이터만 남기기
#df2['상담일지등록일'] = pd.to_datetime(df2['상담일지등록일'],format='%Y-%m-%d')
#df2['상담일지등록일']
#df1['상담일지등록일'] = pd.to_datetime(df1['상담일지등록일'],format='%Y-%m-%d')
#target_month = 1 # 1월
#df.query('상담일지등록일.dt.month == @target_month') ## 또는 df[df['Date'].dt.month==target_month]